<a href="https://colab.research.google.com/github/jmeugniot/bsan6070/blob/main/CA02/CA02_NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Naive Bayes
###Email Spam Classification - CA02
####Jamie Meugniot

In [2]:
##import needed packages to complete Naive Bayes supervised machine learning algorithm
import os #allows you to access a filesystem
import numpy as np
from collections import Counter
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


####Clean & Prep the data for the model
#####Remove all non-required words (words that do not contribute to the analysis - aka Stop Words), espessions, and symbols from the text. 

In [3]:
#Create dictionary of 3000 most common words from all email content
##Add all words and sympbols in the dictionary
##Remove all non-alpha-numeric characters and any single character alpha-numeric characters
##Shrink the dictionary keeping only 3000 most common words 


def create_dict (root_dir): #define new function as "create_dict"
  total_words = [] #create empty list
##construct an absolute path and join path components
#"f" represents an absolute path, so all previous components of the path will be removed and the joining will continue
#os.listdir will return a list containing the names of the entries in the directory given by the path
  path = [os.path.join(root_dir,f) for f in os.listdir(root_dir)]
  for email in path: #create for loop
      with open(email) as m: #using 'with' means there is no need to call file.close()
        for line in m:
          words = line.split() #spliting strings into separate words
          total_words += words ##add split words into the empty list total_words
  word_dict = Counter(total_words) ##counter used to count the words saved in list total_words and saved in word_dict
  words_to_remove = list(word_dict) #list() output returns a list of the interable's items in this case the words in word_dict


  for item in words_to_remove: #create a for loop using the list from the previous step 'words_to_remove'
  #create if statement - if the word in words_to_remove is not all alphabet letters then delete
    if item.isalpha() == False: #isalpha will return a True or False if all characters of a value are alphabet letters
      del word_dict[item] #delete the words that include non alphabet letters from word_dict
  #use the len() to count the number of characters in a given word
    elif len(item) == 1: #if the word is 1 character
      del word_dict[item] #delete the words that equal 1 character from the word_dict
  #words removed from word_dict
  #.most_common will return ordered pairs of the word and its corresponding count in word_dict
  word_dict = word_dict.most_common(3000) #set parameter of 3000 ordered pairs
  return word_dict

 

This function extracts feature columns and populates their values (Feature Matrix of 3000 comumns and rows equal to the number of email files). The function also analyzes the File Names of each email file and decides if it's a Spam or not based on the naming convention. Based on this the function also creates the Labelled Data Column. This function is used to extract the training dataset as well as the testing dataset and returns the Feature Dataset and the Label column.

In [4]:
  def extract_features(mail_dir):
  #os.listdir will return a list containing the names of the entries in the directory given by the path
    files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)] #create list of files
  ##create matrix of zeros using np.zeros. Set number of rows equal to the total files and the columns equal to the 3000 most commmon words
    features_matrix = np.zeros((len(files),3000))
  ##create 1 column of zeros equal to the total number of files
    train_labels = np.zeros(len(files))
    count = 1; #set count starting number
    docID = 0; #set docID starting at 0
    for emails in files: #iterate over each email file saved in the files path
      with open(emails) as fi: #use "fi" to refer to the file object
        for i, line in enumerate(fi): #enumerate counts the number of lines in the email
          if i ==2: ##starting at the 3rd line which is were the email message starts 
            words = line.split() ##split the line of text into a list of words and store in list 'words'
            for word in words: ##iterate over each word in the list of 'words'
              wordID = 0 ##set wordID=0 for each word
              for i, d in enumerate(word_dict):#enumerate is counting the number of words in the word_dict
                if d[0] == word: #if d index 0 is equal to the word
                  wordID = i #set wordID = i 
                  features_matrix[docID,wordID] = words.count(word)
        train_labels[docID] = 0;
        filepathTokens = emails.split('/') #split the email file name at each character '/'
        lastToken = filepathTokens[len(filepathTokens)-1] #set lastToken as the the length of the file name after separation minus 1 - this gives us the final file name
        if lastToken.startswith("spmsg"): #if the file name starts with "spmsg"
          train_labels[docID] = 1; #docID set to 1 to code spam files
          count = count + 1 #increase count by 1 for each iteration
        docID = docID + 1 #increase docID by 1 for each iteration
    return features_matrix, train_labels  

####Main Program
#####Call the above functions 'create_dict' and 'extract_features'. First it will train the model using the training dataset. After that it scores the test data set by running the trained model with the test data set. Final output will be the accuracy score of the model.

In [5]:
##read in both data folders
TRAIN_DIR = "/content/drive/My Drive/MSBA_Colab_2020/ML_Algorithms/CA02/Data/train-mails"
TEST_DIR = "/content/drive/My Drive/MSBA_Colab_2020/ML_Algorithms/CA02/Data/test-mails"

In [7]:
word_dict = create_dict(TRAIN_DIR) #use create_dict function and training data

print ("reading and processing emails from TRAIN and TEST folders") #print the text in quotes
features_matrix, labels = extract_features(TRAIN_DIR) #create matrix using training data
test_features_matrix, test_labels = extract_features(TEST_DIR) #create matrix using test data

model = GaussianNB() #select the model to be used

print ("Training Model using Gaussian Naibe Bayes algorithm .....")
#train the model
model.fit(features_matrix, labels) #fit function adjusts weights according to data values to that better accuracy can be achived
print ("Training completed") #training is complete

print ("testing trained model to predict Test Data labels")
predicted_labels = model.predict(test_features_matrix) #predict function takes one argument and returns the learned label for each object
print ("Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:")
print (accuracy_score(test_labels, predicted_labels)) ##accuracy score calculated based on test_labels matching predicted_labels

reading and processing emails from TRAIN and TEST folders
Training Model using Gaussian Naibe Bayes algorithm .....
Training completed
testing trained model to predict Test Data labels
Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:
0.9653846153846154
